In [1]:
import os

# This guide can only be run with the jax backend.
os.environ["KERAS_BACKEND"] = "jax"

import jax

# We import TF so we can use tf.data.
import tensorflow as tf
import keras
import numpy as np

2024-01-05 07:44:34.398990: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 07:44:34.399035: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 07:44:34.400140: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def get_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x1 = keras.layers.Dense(64, activation="relu")(inputs)
    x2 = keras.layers.Dense(64, activation="relu")(x1)
    outputs = keras.layers.Dense(10, name="predictions")(x2)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = get_model()

# Prepare the training dataset.
batch_size = 32
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784)).astype("float32")
x_test = np.reshape(x_test, (-1, 784)).astype("float32")
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# train_dataset = train_dataset.shuffle(buffer_size=64).batch(batch_size)
train_dataset = train_dataset.batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [3]:
# Instantiate a loss function.
loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

In [4]:
def compute_loss_and_updates(trainable_variables, non_trainable_variables, x, y):
    y_pred, non_trainable_variables = model.stateless_call(
        trainable_variables, non_trainable_variables, x
    )
    print(y_pred)
    loss = loss_fn(y, y_pred)
    return loss, non_trainable_variables


In [5]:
grad_fn = jax.value_and_grad(compute_loss_and_updates, has_aux=True)

In [6]:
def train_step(state, data):
    trainable_variables, non_trainable_variables, optimizer_variables = state
    x, y = data
    (loss, non_trainable_variables), grads = grad_fn(
        trainable_variables, non_trainable_variables, x, y
    )
    trainable_variables, optimizer_variables = optimizer.stateless_apply(
        grads, trainable_variables, optimizer_variables
    )
    # Return updated state
    return loss, (
        trainable_variables,
        non_trainable_variables,
        optimizer_variables,
    )

In [7]:
optimizer.build(model.trainable_variables)

trainable_variables = model.trainable_variables
non_trainable_variables = model.non_trainable_variables
optimizer_variables = optimizer.variables
state = trainable_variables, non_trainable_variables, optimizer_variables

# Training loop
for step, data in enumerate(train_dataset):
    data = (data[0].numpy(), data[1].numpy())
    loss, state = train_step(state, data)
    # Log every 100 batches.
    if step % 100 == 0:
        print(f"Training loss (for 1 batch) at step {step}: {float(loss):.4f}")
        print(f"Seen so far: {(step + 1) * batch_size} samples")

Traced<ConcreteArray([[  31.524109    135.56967      -1.3483505    13.247395    -21.111664
    13.21363     -17.077835   -193.62439     -23.242966     49.782784  ]
 [ 120.16257     129.38353     174.23958     -71.56448    -177.88203
   115.17963      60.960014   -133.63116     -58.19822      14.002335  ]
 [  30.426115     -5.1504784   -47.540955      0.6190672   -82.51708
    22.109928    -11.260504      3.5780945    34.494873     17.191149  ]
 [  69.17001     -33.76323     -86.93546     -41.494022    -42.44924
    34.626938    -47.154015    -61.131073    -10.318655     -4.7830553 ]
 [ 102.8268       28.495008    -67.533554    -54.241997    -59.45101
    44.710487     20.848019    -50.22388     -82.16635      22.092155  ]
 [  13.620922    -13.641014   -142.45761     -66.45045      85.74654
    35.61003    -107.85919    -145.10628     -78.96484      54.67764   ]
 [  51.729397     30.57917      24.945047    -35.59692    -148.33936
    23.759144     68.97758     -28.783669    -88.26081   

ValueError: Argument `optimizer_variables` must be a list of tensors corresponding 1:1 to Adam().variables. Received list with length 6, but expected 14 variables.